In [12]:
import os
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from sklearn import model_selection as sk_model_selection

In [13]:
# from utils import load_dicom_image, load_dicom_images_3d

In [14]:
# train_data_directory = 'D:/brain-classification-data/train'
# data_directory = 'D:/brain-classification-data'

In [15]:
# IMAGE_SIZE = 128
# NUM_IMAGES = 64

In [16]:
# a = load_dicom_images_3d("00000", num_imgs=NUM_IMAGES, img_size=IMAGE_SIZE)
# print(a.shape)
# print(np.min(a), np.max(a), np.mean(a), np.median(a))
# image = a[0]
# print("Dimension of the CT scan is:", image.shape)
# plt.imshow(np.squeeze(image[:, :, 15]), cmap="gray")

In [17]:
train_df = pd.read_csv('data/train_labels.csv')
train_df.head()

,BraTS21ID,MGMT_value
0,0,1
1,2,1
2,3,0
3,5,1
4,6,1


In [18]:
from train import train_mri_type

In [19]:
df_train, df_valid = sk_model_selection.train_test_split(
    train_df, 
    test_size=0.15, 
    random_state=42, 
    stratify=train_df["MGMT_value"],
)

len(df_valid)

88

In [20]:
from models import SimpleCNNModel

In [21]:
model = SimpleCNNModel()

In [22]:
train_mri_type(df_train, df_valid, model, mri_type='FLAIR')

(497, 3) (88, 3)
SimpleCNNModel(
  (conv1): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn4): 

RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 4.00 GiB total capacity; 2.05 GiB already allocated; 0 bytes free; 3.03 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF